## Tic-Tac-Toe Robots
This notebook will simulate many games played by 2 robots each picking completely random moves. It will then save all board states to determine which board states are most likely to win for x or o (or tie).

In [1]:
from collections import defaultdict
import random
import numpy as np
import time
import json
import os

In [2]:
def init_boards():
    x_plays = defaultdict(int)
    o_plays = defaultdict(int)
    return x_plays, o_plays

In [3]:
def starting_board():
    return '_________'

In [4]:
def save_board(game, board):
    game.append(board)    

In [5]:
def random_move(board, player, x_plays, o_plays, num_games, max_games):
    randomness = max(1-num_games/max_games,.1)
    if random.random() < randomness:
        valid_pos = False
        while not valid_pos:
            pos = random.randint(0,8)
            if board[pos] == '_' and pos>=0: 
                return add_move(board, pos, player)
    else:
        valid_moves = []
        inds = []
        for i in range(0,9):
            if board[i] == '_':
                new_board = list(board)#.copy()
                new_board[i] = player
                new_board = ''.join(new_board)
                valid_moves.append(new_board)
                inds.append(i)
        best_moves = []
        d = o_plays.copy()
        if player == 'x':
            d = x_plays.copy()
        for move in valid_moves:
            best_moves.append(d[move])
        return add_move(board, inds[best_moves.index(max(best_moves))], player)

In [6]:
def add_move(board,pos, player):
    l = list(board)
    l[pos] = player
    return ''.join(l)

In [7]:
def check_winner(board, player):
    check = list(board)
    if len(set(check[0:3])) == 1 and check[0] == player: #1st row
        return True
    if len(set(check[3:6])) == 1 and check[3] == player: #2nd row
        return True   
    if len(set(check[6:])) == 1 and check[6] == player: #3rd row
        return True   
    if len(set(check[0::3])) == 1 and check[0] == player: #1st col
        return True
    if len(set(check[1::3])) == 1 and check[1] == player: #2nd col
        return True   
    if len(set(check[2::3])) == 1 and check[2] == player: #3rd col
        return True  
    diag1 = check[0]+check[4]+check[-1]
    diag2 = check[2]+check[4]+check[6]
    if len(set(diag1)) == 1 and check[4] == player: #1st diag
        return True     
    if len(set(diag2)) == 1 and check[4] == player: #2nd diag
        return True   
    return False

In [8]:
def display_board(board):
    disp = np.array(list(board))
    disp.shape = (3,3)
    for n in range(0,9):
        i = int(n/3)
        j = n%3
        disp[i,j] = board[n]
    print(np.where(disp=='_', ' ', disp))

In [9]:
def robots_only(x_plays, o_plays, num_games, max_games):
    player = 'x' #x always goes first
    winner = False
    board = starting_board()
    board_states = [board]
    
    while (not winner) and '_' in board:
        board = random_move(board, player, x_plays, o_plays, num_games, max_games)       
            
        winner = check_winner(board, player)
        if winner == False:
            if player == 'x':player = 'o'
            else: player = 'x'
        board_states.append(board)
   
    if winner == True:
        return board_states, player
    else: 
        return board_states, 'tie'

In [10]:
def save_states(boards, x_plays, o_plays, winner):
    winning_reward = 10
    losing_reward = -20
    tying_reward = 1
    if winner == 'x':
        x_reward = winning_reward
        o_reward = losing_reward
    elif winner == 'o':
        x_reward = losing_reward 
        o_reward = winning_reward       
    else:
        x_reward = o_reward = tying_reward
    for board, i in zip(boards[::2], range(len(boards))):
        x_plays[board] = x_plays[board]*.9
        x_plays[board] += .1*x_reward * (1-(len(boards)*4+i*4)/100)
    for board in boards[1::2]:
        o_plays[board] = o_plays[board]*.9 
        o_plays[board] += .1*o_reward * (1-(len(boards)*4+i*4)/100)

In [11]:
player = 'x' #x always goes first
winner = False
board = starting_board()
x_plays, o_plays = init_boards()

random.seed(7242)

start_time = time.time()

num_games = 0
n = 2500000
while num_games < n:
    boards, winner = robots_only(x_plays, o_plays, num_games, n)
    save_states(boards[1:], x_plays, o_plays, winner)
    num_games+= 1
    if num_games % 50000 == 0: print(num_games, round(time.time() - start_time, 1))
    
print(f"{n} games played in {round(time.time() - start_time, 1)} seconds.")

50000 2.5
100000 5.8
150000 10.0
200000 14.8
250000 20.2
300000 26.9
350000 34.5
400000 42.8
450000 51.8
500000 61.7
550000 72.2
600000 83.5
650000 95.7
700000 108.7
750000 122.3
800000 136.7
850000 151.9
900000 167.8
950000 184.4
1000000 201.9
1050000 220.7
1100000 240.0
1150000 262.1
1200000 283.4
1250000 305.3
1300000 329.4
1350000 353.7
1400000 379.4
1450000 405.6
1500000 434.1
1550000 460.9
1600000 488.6
1650000 517.4
1700000 547.1
1750000 577.8
1800000 611.4
1850000 644.2
1900000 678.1
1950000 713.1
2000000 749.7
2050000 787.0
2100000 826.2
2150000 866.7
2200000 908.3
2250000 951.4
2300000 995.7
2350000 1039.1
2400000 1082.3
2450000 1128.7
2500000 1174.7
2500000 games played in 1174.7 seconds.


In [12]:
script_dir = os.path.dirname('__file__') #<-- absolute dir the script is in

with open(os.path.join(script_dir, './x_plays/x_plays.txt'), 'w') as file:
     file.write(json.dumps(x_plays))
with open(os.path.join(script_dir, './o_plays/o_plays.txt'), 'w') as file:
     file.write(json.dumps(o_plays))
 